In [ ]:
import plotly.graph_objects as go
import pandas as pd
import yfinance as yf
import time
import numpy as np
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from IPython.display import clear_output, display

def get_eurcad_data(interval="5m", period="60d"):
    try:
        eurcad = yf.Ticker("EURCAD=X")
        data = eurcad.history(period=period, interval=interval)
        return data
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

def feature_engineering(data):
    data['Return'] = data['Close'].pct_change()
    data['SMA_9'] = data['Close'].rolling(window=9).mean()
    data['SMA_18'] = data['Close'].rolling(window=18).mean()
    data['Volatility'] = data['Close'].rolling(window=20).std()
    data['High_Low_Range'] = data['High'] - data['Low']
    data['Breakout'] = (data['Close'] > data['Close'].rolling(window=20).max()).astype(int)
    data['Breakout_Fade'] = (data['Close'] < data['Close'].rolling(window=10).min()).astype(int)
    data.dropna(inplace=True)
    return data

def prepare_ml_data(data):
    feature_columns = ['Return', 'SMA_9', 'SMA_18', 'Volatility', 'High_Low_Range', 'Breakout', 'Breakout_Fade']
    X = data[feature_columns]
    y = np.where(data['Close'].shift(-1) > data['Close'], 1, 0)  # 1 for Buy, 0 for Sell
    return X, y

def train_ml_model():
    data = get_eurcad_data()
    if data is None:
        return None
    data = feature_engineering(data)
    X, y = prepare_ml_data(data)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    accuracy = model.score(X_test, y_test)
    print(f"Model trained with accuracy: {accuracy:.2f}")
    return model, scaler

def analyze_and_generate_signals(data, model, scaler):
    if data is None or data.empty or len(data) < 20:
        return None, None
    
    data = feature_engineering(data)
    X_latest = data.iloc[-1:][['Return', 'SMA_9', 'SMA_18', 'Volatility', 'High_Low_Range', 'Breakout', 'Breakout_Fade']]
    X_latest = scaler.transform(X_latest)
    prediction = model.predict(X_latest)[0]
    
    signal_type = "CALL 🟩" if prediction == 1 else "PUT 🟥"
    current_time = datetime.utcnow() - timedelta(hours=3)
    trade_time = (current_time + timedelta(minutes=5)).strftime("%H:%M")
    signal_text = f"💰 5 minutes expiry\nEUR/CAD;{trade_time};{signal_type}\n🕐 TIME TO {trade_time}"
    return signal_text, trade_time

def create_candlestick_chart(data, title):
    if data is None or data.empty:
        print(f"No data to display for {title}.")
        return None
    
    fig = go.Figure(data=[go.Candlestick(x=data.index,
                                         open=data['Open'],
                                         high=data['High'],
                                         low=data['Low'],
                                         close=data['Close'])])
    fig.update_layout(title=title, xaxis_title='Time', yaxis_title='Price', xaxis_rangeslider_visible=False)
    return fig

def update_charts_and_signals():
    model, scaler = train_ml_model()
    if model is None:
        print("Model training failed.")
        return
    
    signals_history = []
    try:
        while True:
            data_5min = get_eurcad_data(interval="5m")
            fig_5min = create_candlestick_chart(data_5min, "EUR/CAD 5-Minute Chart")
            
            if fig_5min is not None:
                signal_text, trade_time = analyze_and_generate_signals(data_5min, model, scaler)
                
                if signal_text:
                    signals_history.append(signal_text)
                    signals_history = signals_history[-5:]
                    clear_output(wait=True)
                    display(fig_5min)
                    print("Trading Signals:")
                    for signal in signals_history:
                        print(signal)
                else:
                    clear_output(wait=True)
                    display(fig_5min)
                    print("No New Trading Signals")
            
            time.sleep(600)  # 10-minute interval
    except KeyboardInterrupt:
        print("Chart update stopped.")

if __name__ == "__main__":
    update_charts_and_signals()


Trading Signals:
💰 5 minutes expiry
EUR/CAD;04:45;PUT 🟥
🕐 TIME TO 04:45
💰 5 minutes expiry
EUR/CAD;04:55;PUT 🟥
🕐 TIME TO 04:55
💰 5 minutes expiry
EUR/CAD;05:05;PUT 🟥
🕐 TIME TO 05:05
💰 5 minutes expiry
EUR/CAD;05:15;PUT 🟥
🕐 TIME TO 05:15
